# Capítulo 4: Spark SQL

In [413]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.{functions => F}
import org.apache.spark.sql._
import org.apache.spark.sql.types.DateType
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions.{asc, col, desc}
case class AverageDelay (origin: String, averageDelay: Double)

import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.{functions=>F}
import org.apache.spark.sql._
import org.apache.spark.sql.types.DateType
import org.apache.spark.sql.Dataset
import org.apache.spark.sql.functions.{asc, col, desc}
defined class AverageDelay


In [156]:
val spark = SparkSession.builder.appName("Ejemplo").getOrCreate()
import spark.implicits._

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@64fd0ec4
import spark.implicits._


#### Observemos que guardamos ds como un DataSet de acuerdo con la clase definida anteriormente

In [195]:
val df = spark.read.format("csv").option("inferSchema","true").option("header","true").load("departuredelays.csv")
val ds = df.groupBy(col("origin")).agg(F.avg(col("delay")) as "averageDelay").as[AverageDelay]
ds.show(5)
df.orderBy(desc("date")).show(5, truncate=false)

+------+------------------+
|origin|      averageDelay|
+------+------------------+
|   GEG| 6.068493150684931|
|   BUR| 8.316794644615081|
|   GRB|   8.4463480613165|
|   GTF| 4.741176470588235|
|   GRR|15.304448742746615|
+------+------------------+
only showing top 5 rows

+-------+-----+--------+------+-----------+
|date   |delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|3312359|-1   |2090    |ANC   |DEN        |
|3312359|0    |1628    |SEA   |IAH        |
|3312359|6    |1604    |SFO   |ORD        |
|3312359|5    |1501    |SLC   |CLT        |
|3312359|39   |1859    |SFO   |ATL        |
+-------+-----+--------+------+-----------+
only showing top 5 rows



df: org.apache.spark.sql.DataFrame = [date: int, delay: int ... 3 more fields]
ds: org.apache.spark.sql.Dataset[AverageDelay] = [origin: string, averageDelay: double]


#### Creamos una view temporal de cada conjunto de datos. Como vemos no se encuentra ningún conflicto al crear la view desde un DataSet

In [379]:
df.createOrReplaceTempView("us_delay_flights_tbl")
ds.createOrReplaceTempView("delayAverageByOrigin")
df.printSchema()

root
 |-- date: integer (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)



In [452]:
 //spark.sql("""SELECT origin, delay FROM us_delay_flights_tbl WHERE origin LIKE "%Y%" GROUP BY origin ORDER BY origin desc""").show(5)//Sirve tanto "..." como """..."""
//pero observemos que el modo """...""" tiene su virtud, al poder introducir "..." dentro.

#### Y hacemos una prueba con un par de consultas sencillas, de manera que siguiendo caminos distintos llegamos al mismo resultado

In [200]:
spark.sql("SELECT origin, AVG(delay) as averageDelay FROM us_delay_flights_tbl GROUP BY origin ORDER BY averageDelay asc").show(5)
spark.sql("""SELECT origin, averageDelay FROM delayAverageByOrigin ORDER BY averageDelay asc""").show(5)

+------+-------------------+
|origin|       averageDelay|
+------+-------------------+
|   YAK| -6.685393258426966|
|   CDV| -5.752808988764045|
|   GFK| -5.333333333333333|
|   FAI|  -2.95935960591133|
|   OTZ|-2.8813559322033897|
+------+-------------------+
only showing top 5 rows

+------+-------------------+
|origin|       averageDelay|
+------+-------------------+
|   YAK| -6.685393258426966|
|   CDV| -5.752808988764045|
|   GFK| -5.333333333333333|
|   FAI|  -2.95935960591133|
|   OTZ|-2.8813559322033897|
+------+-------------------+
only showing top 5 rows



#### Nos encontramos con una bondad de usar Spark SQL, y es que toda consulta hecha con spark.sql("consulta") devuelve un DataFrame

In [151]:
val a = spark.sql("""SELECT COUNT(averageDelay) FROM delayAverageByOrigin WHERE averageDelay > -0.5 AND averageDelay < 0.5""")

a: org.apache.spark.sql.DataFrame = [count(averageDelay): bigint]


In [152]:
val b =ds.where(col("averageDelay") > -0.5 && col("averageDelay") < 0.5).count()

b: Long = 6


#### Lo verificamos en las anteriores consultas, equivalentes pero con una salida en forma de DataFrame en el caso de usar Spark SQL y un Long en otro caso. Verificamos que efectivamente la consulta da como resultado lo mismo.

In [153]:
a.show()
print(b)

+-------------------+
|count(averageDelay)|
+-------------------+
|                  6|
+-------------------+

6

#### Al ser objetos distintos la forma de mostrar por pantalla también dista de ser igual. Parece lógico pensar que que esto podría tener algún impacto en la eficiencia.
#### Probemos con consultas más complejas

In [471]:
spark.sql("""SELECT date, delay, origin, destination  
FROM us_delay_flights_tbl  
WHERE delay > 120 AND ORIGIN = 'SFO' AND DESTINATION = 'ORD'  
ORDER by delay DESC""").show(10)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|2190925| 1638|   SFO|        ORD|
|1031755|  396|   SFO|        ORD|
|1022330|  326|   SFO|        ORD|
|1051205|  320|   SFO|        ORD|
|1190925|  297|   SFO|        ORD|
|2171115|  296|   SFO|        ORD|
|1071040|  279|   SFO|        ORD|
|1051550|  274|   SFO|        ORD|
|3120730|  266|   SFO|        ORD|
|1261104|  258|   SFO|        ORD|
+-------+-----+------+-----------+
only showing top 10 rows



In [472]:
spark.sql("""SELECT delay, origin, destination, 
 CASE 
 WHEN delay > 360 THEN 'Very Long Delays' 
 WHEN delay > 120 AND delay < 360 THEN 'Long Delays' 
 WHEN delay > 60 AND delay < 120 THEN 'Short Delays' 
 WHEN delay > 0 and delay < 60 THEN 'Tolerable Delays' 
 WHEN delay = 0 THEN 'No Delays' 
 ELSE 'Early' 
 END AS Flight_Delays 
 FROM us_delay_flights_tbl 
 ORDER BY origin, delay DESC""").show(10)

+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  Long Delays|
|  305|   ABE|        ATL|  Long Delays|
|  275|   ABE|        ATL|  Long Delays|
|  257|   ABE|        ATL|  Long Delays|
|  247|   ABE|        DTW|  Long Delays|
|  247|   ABE|        ATL|  Long Delays|
|  219|   ABE|        ORD|  Long Delays|
|  211|   ABE|        ATL|  Long Delays|
|  197|   ABE|        DTW|  Long Delays|
|  192|   ABE|        ORD|  Long Delays|
+-----+------+-----------+-------------+
only showing top 10 rows



#### Procedemos a tratar la fecha

Se ha observado que 0111 es leido como 111 así que sumamos lo necesario para conseguir 10111

In [545]:
spark.sql("""SELECT delay, distance, origin, destination,cast((date+100000000) as char(30)) as date FROM us_delay_flights_tbl""").createOrReplaceTempView("us_delay_flights_tblAUX")
spark.sql("""SELECT * FROM us_delay_flights_tblAUX""").show()

+-----+--------+------+-----------+---------+
|delay|distance|origin|destination|     date|
+-----+--------+------+-----------+---------+
|    6|     602|   ABE|        ATL|101011245|
|   -8|     369|   ABE|        DTW|101020600|
|   -2|     602|   ABE|        ATL|101021245|
|   -4|     602|   ABE|        ATL|101020605|
|   -4|     602|   ABE|        ATL|101031245|
|    0|     602|   ABE|        ATL|101030605|
|   10|     602|   ABE|        ATL|101041243|
|   28|     602|   ABE|        ATL|101040605|
|   88|     602|   ABE|        ATL|101051245|
|    9|     602|   ABE|        ATL|101050605|
|   -6|     602|   ABE|        ATL|101061215|
|   69|     602|   ABE|        ATL|101061725|
|    0|     369|   ABE|        DTW|101061230|
|   -3|     602|   ABE|        ATL|101060625|
|    0|     369|   ABE|        DTW|101070600|
|    0|     602|   ABE|        ATL|101071725|
|    0|     369|   ABE|        DTW|101071230|
|    0|     602|   ABE|        ATL|101070625|
|    0|     569|   ABE|        ORD

In [548]:
//Por qué no me deja guardarlo como "us_delay_flights_tblAUX"?
spark.sql("""SELECT date, SUBSTRING(date, 2,2) as month,SUBSTRING(date, 4,2) as day, SUBSTRING(date, 6,2) as hour, SUBSTRING(date, 8,2) as min
FROM us_delay_flights_tblAUX""").createOrReplaceTempView("us_delay_flights_tblAUX1")

#### Éxito al convertir la fecha

In [549]:
spark.sql("""SELECT * FROM us_delay_flights_tblAUX1""").show()

+---------+-----+---+----+---+
|     date|month|day|hour|min|
+---------+-----+---+----+---+
|101011245|   01| 01|  12| 45|
|101020600|   01| 02|  06| 00|
|101021245|   01| 02|  12| 45|
|101020605|   01| 02|  06| 05|
|101031245|   01| 03|  12| 45|
|101030605|   01| 03|  06| 05|
|101041243|   01| 04|  12| 43|
|101040605|   01| 04|  06| 05|
|101051245|   01| 05|  12| 45|
|101050605|   01| 05|  06| 05|
|101061215|   01| 06|  12| 15|
|101061725|   01| 06|  17| 25|
|101061230|   01| 06|  12| 30|
|101060625|   01| 06|  06| 25|
|101070600|   01| 07|  06| 00|
|101071725|   01| 07|  17| 25|
|101071230|   01| 07|  12| 30|
|101070625|   01| 07|  06| 25|
|101071219|   01| 07|  12| 19|
|101080600|   01| 08|  06| 00|
+---------+-----+---+----+---+
only showing top 20 rows



#### Creamos una tabla conjunta suponiendo que la tabla corresponde a los vuelos de 2020

In [604]:
spark.sql("""SELECT cast(SUBSTRING(date, 2,2) as INTEGER) as month, cast(SUBSTRING(date, 4,2) as INTEGER) as day, cast(SUBSTRING(date, 6,2) as INTEGER) as hour, cast(SUBSTRING(date, 8,2) as INTEGER) as minute,
concat("20/",cast(SUBSTRING(date, 2,2) as INTEGER),"/",cast(SUBSTRING(date, 4,2) as INTEGER)," ", cast(SUBSTRING(date, 6,2) as INTEGER),":", cast(SUBSTRING(date, 8,2) as INTEGER),":00") as DATE, delay, distance, origin, destination
FROM us_delay_flights_tblAUX""").createOrReplaceTempView("us_delay_flights_tblAUX2")
spark.sql("""SELECT * FROM us_delay_flights_tblAUX2""").show()

+-----+---+----+------+---------------+-----+--------+------+-----------+
|month|day|hour|minute|           DATE|delay|distance|origin|destination|
+-----+---+----+------+---------------+-----+--------+------+-----------+
|    1|  1|  12|    45|20/1/1 12:45:00|    6|     602|   ABE|        ATL|
|    1|  2|   6|     0|  20/1/2 6:0:00|   -8|     369|   ABE|        DTW|
|    1|  2|  12|    45|20/1/2 12:45:00|   -2|     602|   ABE|        ATL|
|    1|  2|   6|     5|  20/1/2 6:5:00|   -4|     602|   ABE|        ATL|
|    1|  3|  12|    45|20/1/3 12:45:00|   -4|     602|   ABE|        ATL|
|    1|  3|   6|     5|  20/1/3 6:5:00|    0|     602|   ABE|        ATL|
|    1|  4|  12|    43|20/1/4 12:43:00|   10|     602|   ABE|        ATL|
|    1|  4|   6|     5|  20/1/4 6:5:00|   28|     602|   ABE|        ATL|
|    1|  5|  12|    45|20/1/5 12:45:00|   88|     602|   ABE|        ATL|
|    1|  5|   6|     5|  20/1/5 6:5:00|    9|     602|   ABE|        ATL|
|    1|  6|  12|    15|20/1/6 12:15:00

###### Observemos que puede ser interesante combinar las dos formas de consultar. Por ejemplo en este caso para contar el número de vuelos en un preciso instante.

In [615]:
spark.sql("""SELECT * FROM us_delay_flights_tblAUX2 WHERE month == 1 AND day == 1 AND hour==12 AND minute==45""").show()
spark.sql("""SELECT * FROM us_delay_flights_tblAUX2 WHERE month == 1 AND day == 1 AND hour==12 AND minute==45""").count()

+-----+---+----+------+---------------+-----+--------+------+-----------+
|month|day|hour|minute|           DATE|delay|distance|origin|destination|
+-----+---+----+------+---------------+-----+--------+------+-----------+
|    1|  1|  12|    45|20/1/1 12:45:00|    6|     602|   ABE|        ATL|
|    1|  1|  12|    45|20/1/1 12:45:00|   22|     636|   ATL|        DFW|
|    1|  1|  12|    45|20/1/1 12:45:00|   -4|     505|   ATL|        FLL|
|    1|  1|  12|    45|20/1/1 12:45:00|   -1|     662|   ATL|        LGA|
|    1|  1|  12|    45|20/1/1 12:45:00|    3|     633|   BOS|        CLT|
|    1|  1|  12|    45|20/1/1 12:45:00|   -4|     819|   BOS|        BNA|
|    1|  1|  12|    45|20/1/1 12:45:00|   32|     311|   BUR|        SMF|
|    1|  1|  12|    45|20/1/1 12:45:00|   14|     257|   BUR|        SJC|
|    1|  1|  12|    45|20/1/1 12:45:00|   -3|     355|   BWI|        DTW|
|    1|  1|  12|    45|20/1/1 12:45:00|    3|     321|   BWI|        BOS|
|    1|  1|  12|    45|20/1/1 12:45:00

res566: Long = 52


#### Vayamos a consulta propuesta en el libro

Nos pide ver en qué época del año hay mayor delay. Parece lógico juntar todo lo visto hasta ahora. Por un lado las fechas bien puestas y por 
el otro lado una etiqueta a cada delay. Así podremos agrupar por tipo de delay y contar el número de tipos de delay. Para sacar así una conclusión.

In [642]:
spark.sql("""SELECT cast(SUBSTRING(date, 2,2) as INTEGER) as month, cast(SUBSTRING(date, 4,2) as INTEGER) as day, cast(SUBSTRING(date, 6,2) as INTEGER) as hour, cast(SUBSTRING(date, 8,2) as INTEGER) as minute,
concat("20/",cast(SUBSTRING(date, 2,2) as INTEGER),"/",cast(SUBSTRING(date, 4,2) as INTEGER)," ", cast(SUBSTRING(date, 6,2) as INTEGER),":", cast(SUBSTRING(date, 8,2) as INTEGER),":00") as DATE, delay,CASE 
 WHEN delay > 360 THEN 'Very Long Delays' 
 WHEN delay > 120 AND delay < 360 THEN 'Long Delays' 
 WHEN delay > 60 AND delay < 120 THEN 'Short Delays' 
 WHEN delay > 0 and delay < 60 THEN 'Tolerable Delays' 
 WHEN delay = 0 THEN 'No Delays' 
 ELSE 'Early' 
 END AS Flight_Delays  ,distance, origin, destination
FROM us_delay_flights_tblAUX""").createOrReplaceTempView("us_delay_flights_tblAUX3")
spark.sql("""SELECT * FROM us_delay_flights_tblAUX3""").show()

+-----+---+----+------+---------------+-----+----------------+--------+------+-----------+
|month|day|hour|minute|           DATE|delay|   Flight_Delays|distance|origin|destination|
+-----+---+----+------+---------------+-----+----------------+--------+------+-----------+
|    1|  1|  12|    45|20/1/1 12:45:00|    6|Tolerable Delays|     602|   ABE|        ATL|
|    1|  2|   6|     0|  20/1/2 6:0:00|   -8|           Early|     369|   ABE|        DTW|
|    1|  2|  12|    45|20/1/2 12:45:00|   -2|           Early|     602|   ABE|        ATL|
|    1|  2|   6|     5|  20/1/2 6:5:00|   -4|           Early|     602|   ABE|        ATL|
|    1|  3|  12|    45|20/1/3 12:45:00|   -4|           Early|     602|   ABE|        ATL|
|    1|  3|   6|     5|  20/1/3 6:5:00|    0|       No Delays|     602|   ABE|        ATL|
|    1|  4|  12|    43|20/1/4 12:43:00|   10|Tolerable Delays|     602|   ABE|        ATL|
|    1|  4|   6|     5|  20/1/4 6:5:00|   28|Tolerable Delays|     602|   ABE|        ATL|

In [659]:
spark.sql("""SELECT month, count(Flight_Delays) as VeryLongDelay FROM us_delay_flights_tblAUX3 WHERE Flight_Delays == "Very Long Delays" GROUP BY month""").show(360)
spark.sql("""SELECT month, count(Flight_Delays) as LongDelay FROM us_delay_flights_tblAUX3 WHERE Flight_Delays == "Long Delays" GROUP BY month""").show(360)
spark.sql("""SELECT month, count(Flight_Delays) as Early FROM us_delay_flights_tblAUX3 WHERE Flight_Delays == "Early" GROUP BY month""").show(360)

+-----+-------------+
|month|VeryLongDelay|
+-----+-------------+
|    1|          735|
|    2|          433|
|    3|          331|
+-----+-------------+

+-----+---------+
|month|LongDelay|
+-----+---------+
|    1|    14058|
|    2|     9533|
|    3|     7836|
+-----+---------+

+-----+------+
|month| Early|
+-----+------+
|    1|207257|
|    2|201005|
|    3|263023|
+-----+------+



#### Como podemos observar no se tiene el fichero completo. Faltan todos los meses desde abril hasta diciembre.

In [656]:
spark.sql("""SELECT * FROM us_delay_flights_tblAUX3 WHERE minute==59""").show()

+-----+---+----+------+----------------+-----+----------------+--------+------+-----------+
|month|day|hour|minute|            DATE|delay|   Flight_Delays|distance|origin|destination|
+-----+---+----+------+----------------+-----+----------------+--------+------+-----------+
|    1|  7|  23|    59| 20/1/7 23:59:00|   14|Tolerable Delays|    1586|   ABQ|        JFK|
|    1| 14|  23|    59|20/1/14 23:59:00|  -17|           Early|    1586|   ABQ|        JFK|
|    1| 21|  23|    59|20/1/21 23:59:00|    0|       No Delays|    1586|   ABQ|        JFK|
|    1| 28|  23|    59|20/1/28 23:59:00|  -20|           Early|    1586|   ABQ|        JFK|
|    1|  2|  23|    59| 20/1/2 23:59:00|   20|Tolerable Delays|    2090|   ANC|        DEN|
|    1|  3|  23|    59| 20/1/3 23:59:00|   36|Tolerable Delays|    2090|   ANC|        DEN|
|    1|  4|  23|    59| 20/1/4 23:59:00| 1033|Very Long Delays|    2090|   ANC|        DEN|
|    1|  5|  23|    59| 20/1/5 23:59:00|   42|Tolerable Delays|    2090|   ANC| 